In [1]:
# Data preprocessing
import pandas as pd
import numpy as np
import os
from parser import Competitions, Events, Matches, Lineups

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Plotting
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline
import seaborn as sns
import scikitplot as skplt
import plotly.graph_objects as go
import networkx as nx

In [4]:
comps = Competitions().get_data()
comps

,competition_gender,competition_id,competition_name,country_name,match_available,match_updated,season_id,season_name
0,female,37,FA Women's Super League,England,2019-06-23T15:32:29.914,2019-06-23T15:32:29.914,4,2018/2019
1,male,43,FIFA World Cup,International,2019-06-23T12:38:35.142,2019-06-23T12:38:35.142,3,2018
2,male,11,La Liga,Spain,2019-07-29T20:44:30.861,2019-07-29T20:44:30.861,27,2015/2016
3,male,11,La Liga,Spain,2019-07-30T12:42:05.563,2019-07-30T12:42:05.563,26,2014/2015
4,male,11,La Liga,Spain,2019-07-24T19:44:48.866,2019-07-24T19:44:48.866,25,2013/2014
5,male,11,La Liga,Spain,2019-07-29T17:46:18.935,2019-07-29T17:46:18.935,24,2012/2013
6,male,11,La Liga,Spain,2019-07-11T16:55:27.568,2019-07-11T16:55:27.568,23,2011/2012
7,male,11,La Liga,Spain,2019-08-01T17:44:54.870,2019-08-01T17:44:54.870,22,2010/2011
8,male,11,La Liga,Spain,2019-07-11T07:44:14.533,2019-07-11T07:44:14.533,21,2009/2010
9,male,11,La Liga,Spain,2019-07-06T22:42:14.468,2019-07-06T22:42:14.468,41,2008/2009


In [5]:
nwsl_matches = Matches(competition_id = '49', season_id = '3').get_data()
fawsl_matches = Matches(competition_id = '37', season_id = '4').get_data()
wwc_matches = Matches(competition_id = '72', season_id = '30').get_data()

In [6]:
wwc_matches.sort_values('match_date').tail(1)

,away_score,away_team.away_team_gender,away_team.away_team_group,away_team.away_team_id,away_team.away_team_name,away_team.country.id,away_team.country.name,away_team.managers,competition.competition_id,competition.competition_name,...,match_status,match_week,metadata.data_version,metadata.shot_fidelity_version,season.season_id,season.season_name,stadium.country.id,stadium.country.name,stadium.id,stadium.name
51,0,female,Group E,851,Netherlands Women's,160.0,Netherlands,NaN,72,Women's World Cup,...,available,7,1.1.0,2,30,2019,NaN,NaN,NaN,NaN


In [7]:
final_passes = Events(match_id='69321').get_action_data(action_type='Pass')

In [8]:
final_passes.columns

Index(['counterpress', 'duration', 'id', 'index', 'location', 'minute',
       'off_camera', 'pass.aerial_won', 'pass.angle', 'pass.assisted_shot_id',
       'pass.body_part.id', 'pass.body_part.name', 'pass.cross',
       'pass.cut_back', 'pass.end_location', 'pass.goal_assist',
       'pass.height.id', 'pass.height.name', 'pass.inswinging', 'pass.length',
       'pass.outcome.id', 'pass.outcome.name', 'pass.outswinging',
       'pass.recipient.id', 'pass.recipient.name', 'pass.shot_assist',
       'pass.straight', 'pass.switch', 'pass.technique.id',
       'pass.technique.name', 'pass.through_ball', 'pass.type.id',
       'pass.type.name', 'period', 'play_pattern.id', 'play_pattern.name',
       'player.id', 'player.name', 'position.id', 'position.name',
       'possession', 'possession_team.id', 'possession_team.name',
       'related_events', 'second', 'team.id', 'team.name', 'timestamp',
       'type.id', 'type.name', 'under_pressure'],
      dtype='object')

In [9]:
test = final_passes.sort_values(['minute', 'second'])
test = test.loc[test['team.name'] == "United States Women's"]

In [10]:
def start_coordinates(row):
    coords = row['location']
    return [coords[0], coords[1]]

def end_coordinates(row):
    coords = row['pass.end_location']
    return [coords[0], coords[1]]

def clean(row):
    if row['pass.recipient.name'] is None:
        return row['player.name']
    return row['pass.recipient.name']

test['start'] = test.apply(start_coordinates, axis = 1)
test['end'] = test.apply(end_coordinates, axis = 1)
test['recipient'] = test.apply(clean, axis = 1)
test['pair'] = test.apply(lambda x: [x['player.name'], x['pass.recipient.name']], axis = 1)

In [11]:
test['pair'].value_counts().head()

[Crystal Alyssia Dunn, Megan Rapinoe]       17
[Becky Sauerbrunn, Crystal Alyssia Dunn]    13
[Megan Rapinoe, Crystal Alyssia Dunn]       11
[Crystal Alyssia Dunn, Samantha Mewis]      11
[Samantha Mewis, Megan Rapinoe]             11
Name: pair, dtype: int64

In [12]:
G = nx.DiGraph()
for player in test['player.name'].unique():
    G.add_node(player)

In [13]:
def edging(value):
    if value in G.edges():
        G.edges()[value[0], value[1]]['weight'] += 1
    else:
        G.add_edge(value[0], value[1], weight = 1)

In [14]:
#G.clear()
#nx.draw(G, with_labels=True, font_weight='bold')

In [15]:
# edgewidth = [ d['weight'] for (u,v,d) in G.edges(data=True)]
# plt.figure(figsize=(20,10))

# pos = nx.circular_layout(G)
# nx.draw_networkx_nodes(G, pos, with_labels = True)
# nx.draw_networkx_edges(G, pos, width=edgewidth)